In [4]:
from ultralytics import YOLO
from pathlib import Path
from PIL import Image
import tkinter as tk
from tkinter import filedialog, messagebox
import os

MODEL = "yolo11n.pt"
CONFIDENCE = 0.25
IMG_SIZE = 640
IMG_EXTS = {'.jpg','.jpeg','.png','.bmp','.tiff'}

print("Loading model:", MODEL)
model = YOLO(MODEL)
print("Model loaded. Model knows these classes:", model.names)


def annotate_folder(folder_path: str, conf: float = CONFIDENCE, imgsz: int = IMG_SIZE):
    """Annotate all images in folder_path using yolo11n.
    Creates two subfolders inside folder_path:
      - annotated/      (annotated images saved as same filename)
      - annotations/    (YOLO-format .txt files, one per image)
    Returns (annotated_dir, ann_dir)
    """
    folder = Path(folder_path)
    if not folder.exists() or not folder.is_dir():
        raise ValueError(f"Not a folder: {folder_path}")

    annotated_dir = folder / "annotated_valid"
    ann_dir = folder / "annotations_valid"
    annotated_dir.mkdir(exist_ok=True)
    ann_dir.mkdir(exist_ok=True)

    imgs = [p for p in sorted(folder.iterdir()) if p.suffix.lower() in IMG_EXTS]
    if len(imgs) == 0:
        print("No images found in", folder)
        return annotated_dir, ann_dir

    for img_path in imgs:
        print("Processing:", img_path.name)
        results = model.predict(source=str(img_path), conf=conf, imgsz=imgsz, verbose=False)
        res = results[0] 
        im = Image.open(img_path)
        width, height = im.size

        yolo_lines = []
        boxes = getattr(res, 'boxes', None)

        if boxes is not None and len(boxes) > 0:

            try:
                xyxyn = boxes.xyxyn.cpu().numpy()
                cls_ids = boxes.cls.cpu().numpy().astype(int)
                confs = boxes.conf.cpu().numpy()
            except Exception:

                xyxy = boxes.xyxy.cpu().numpy()
                cls_ids = boxes.cls.cpu().numpy().astype(int)
                confs = boxes.conf.cpu().numpy()
                xyxyn = xyxy.copy()
                xyxyn[:, [0,2]] /= width
                xyxyn[:, [1,3]] /= height

            for (xmin, ymin, xmax, ymax), cls_id, conf_score in zip(xyxyn, cls_ids, confs):

                xmin = float(max(0.0, min(1.0, xmin)))
                ymin = float(max(0.0, min(1.0, ymin)))
                xmax = float(max(0.0, min(1.0, xmax)))
                ymax = float(max(0.0, min(1.0, ymax)))

                x_center = (xmin + xmax) / 2.0
                y_center = (ymin + ymax) / 2.0
                w_box = xmax - xmin
                h_box = ymax - ymin

                yolo_lines.append(f"{int(cls_id)} {x_center:.6f} {y_center:.6f} {w_box:.6f} {h_box:.6f}")

        txt_path = ann_dir / (img_path.stem + '.txt')
        with open(txt_path, 'w') as f:
            f.write('\n'.join(yolo_lines))

        try:
            ann_img = res.plot()  
            Image.fromarray(ann_img).save(annotated_dir / img_path.name)
        except Exception as e:
            print("Warning: could not plot result for", img_path.name, "- saving original. Error:", e)
            im.save(annotated_dir / img_path.name)

    print("Finished. Annotated images ->", annotated_dir)
    print("YOLO txts ->", ann_dir)
    return annotated_dir, ann_dir


def pick_folder_and_annotate():
    root = tk.Tk()
    root.withdraw()
    folder = filedialog.askdirectory(title='Select images folder to annotate (will create annotated/ and annotations/)')
    if not folder:
        print('No folder selected.')
        return
    try:
        annotate_folder(folder)
        messagebox.showinfo('Done', f'Annotated images and YOLO txts saved inside:\n{folder}')
    except Exception as e:
        messagebox.showerror('Error', str(e))

Loading model: yolo11n.pt
Model loaded. Model knows these classes: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: '

In [3]:
# Run this cell to actually annotate a folder (no GUI dialog)
from pathlib import Path

# Replace this with your images folder path
images_folder = "Downloads/Archive/archive/valid"

# Call the function defined in the replacement cell
annotate_folder(images_folder)

Processing: 000000013182_jpg.rf.a190c0c4e1a6299e8ce517e4ec2b654a.jpg
Processing: 000000014432_jpg.rf.ca4d67886840701676c36b8db4870411.jpg
Processing: 000000015906_jpg.rf.989fa80c5a2705288fea5964511e7ba6.jpg
Processing: 000000018460_jpg.rf.dc22418012e3ebcc78513761d1058994.jpg
Processing: 000000021423_jpg.rf.0718abaffd600d470c3228d10eb7b0bb.jpg
Processing: 000000022801_jpg.rf.cb88bb42f81298da55f169af10b2499e.jpg
Processing: 000000026676_jpg.rf.f84249d914ffb2e5a5ec853fdad8aae8.jpg
Processing: 000000028186_jpg.rf.9e61d73eb51f9246b728ba050e69b59c.jpg
Processing: 000000029519_jpg.rf.6bd693a835670618ff7f174f8f03d573.jpg
Processing: 000000032174_jpg.rf.23533ca642d18122285840d185c57285.jpg
Processing: 000000033571_jpg.rf.de9e08c25405eba31f8e8fb5a0374277.jpg
Processing: 000000036732_jpg.rf.bdd0259f9e7ddf42f5b18deb46b26cda.jpg
Processing: 000000044989_jpg.rf.8ade50d7f653f01dbef003dd4cc616c1.jpg
Processing: 000000045770_jpg.rf.9ff929fe27ae3fbdb75eb1676e5ee76f.jpg
Processing: 000000050518_jpg.rf.e5

(WindowsPath('Downloads/Archive/archive/valid/annotated_valid'),
 WindowsPath('Downloads/Archive/archive/valid/annotations_valid'))

In [7]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from stable_baselines3 import PPO

class CrowdEvacEnv(gym.Env):
    def __init__(self):
        super(CrowdEvacEnv, self).__init__()
        self.observation_space = spaces.Box(low=0, high=1, shape=(5, 5), dtype=np.float32)
        self.action_space = spaces.Discrete(4)  # Up, Down, Left, Right
        self.state = np.zeros((5, 5), dtype=np.float32)
        self.agent_pos = [2, 2]

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.state = np.zeros((5, 5), dtype=np.float32)
        self.state[1, 1] = 1  # Fire location
        self.agent_pos = [2, 2]
        return self.state, {}

    def step(self, action):
        # Move agent according to action but keep within bounds
        if action == 0 and self.agent_pos[0] > 0:
            self.agent_pos[0] -= 1
        elif action == 1 and self.agent_pos[0] < 4:
            self.agent_pos[0] += 1
        elif action == 2 and self.agent_pos[1] > 0:
            self.agent_pos[1] -= 1
        elif action == 3 and self.agent_pos[1] < 4:
            self.agent_pos[1] += 1

        # Reward & done conditions
        terminated = (self.agent_pos == [0, 0]) or (self.state[self.agent_pos[0], self.agent_pos[1]] == 1)
        truncated = False  # No truncation logic
        reward = 10 if self.agent_pos == [0, 0] else -1
        info = {}

        return self.state, reward, terminated, truncated, info

# Instantiate environment
env = CrowdEvacEnv()

# Create PPO model
model = PPO('MlpPolicy', env, verbose=1)

# Train the model
model.learn(total_timesteps=1000)

# Optional: test the trained agent
obs, _ = env.reset()
done = False
while not done:
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    print(f"Action: {action}, Reward: {reward}, Done: {done}")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 39.7     |
|    ep_rew_mean     | -38.1    |
| time/              |          |
|    fps             | 2053     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
Action: 0, Reward: -1, Done: False
Action: 0, Reward: -1, Done: False
Action: 1, Reward: -1, Done: False
Action: 0, Reward: -1, Done: False
Action: 3, Reward: -1, Done: False
Action: 0, Reward: -1, Done: False
Action: 3, Reward: -1, Done: False
Action: 0, Reward: -1, Done: False
Action: 2, Reward: -1, Done: False
Action: 1, Reward: -1, Done: False
Action: 2, Reward: -1, Done: False
Action: 3, Reward: -1, Done: False
Action: 2, Reward: -1, Done: False
Action: 3, Reward: -1, Done: False
Action: 0, Reward: -1, Done: False
Action: 3, Reward: -1, Done: False
Action